## PyTorch Tutorial 08 - Logistic Regression

In [1]:
# Design model (input, output,forward pass)
# Construct loss and optimizer
# Training loop

# - forward pass: compute prediction and loss
# -  backward pass gradients
# -  update weights

In [7]:
import torch
import torch.nn  as nn 
import numpy as np 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [8]:
from sklearn.metrics import accuracy_score,f1_score

In [11]:
#prepare the data

bc = datasets.load_breast_cancer()
x,y = bc.data,bc.target

n_samples,n_features = x.shape
print(f"records: {n_samples}, features: {n_features}")
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1234)

#scale

sc  =StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(-1,1)
y_test = y_test.view(-1,1)
# model
# f = sig(w*x+b)

class LogisticRegressionModel(nn.Module):

    def __init__(self,input_size,output_size) -> None:
        super(LogisticRegressionModel,self).__init__()
        self.linear = nn.Linear(in_features=input_size,out_features=output_size)
    
    def forward(self,x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
n_sample ,n_features = x.shape
model = LogisticRegressionModel(input_size=n_features,output_size=1)


# loss and optimizer

criterion = nn.BCELoss()
lr= 0.001
optimizer = torch.optim.SGD(model.parameters(),lr = lr)
#training loop
num_epoch = 100
for epoch in range(num_epoch):
    #forwards pass
    y_predicted = model(x_train)

    #backward pass
    loss = criterion(y_predicted,y_train)
    loss.backward()

    #updates
    optimizer.step()

    optimizer.zero_grad()

    if epoch%10==0:
        print(f"Epoch: {epoch+1} loss: {loss.item():.4f}")
    
with torch.no_grad():
    y_predicted = model(x_test)

    #y_predicted is sigmoid i.e proba
    y_predicted_cls = y_predicted.round()

    acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f"accuracy: {acc:.4f}")



records: 569, features: 30
Epoch: 1 loss: 0.5531
Epoch: 11 loss: 0.5427
Epoch: 21 loss: 0.5327
Epoch: 31 loss: 0.5232
Epoch: 41 loss: 0.5141
Epoch: 51 loss: 0.5054
Epoch: 61 loss: 0.4971
Epoch: 71 loss: 0.4891
Epoch: 81 loss: 0.4815
Epoch: 91 loss: 0.4741
accuracy: 0.8860
